# 📘 Notebook 5 — ESG Reporting & Visualization Dashboard

In [1]:
# %% ===============================================================
# STEP 0 — ENVIRONMENT SETUP
# ===============================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")

plt.style.use("seaborn-v0_8-whitegrid")
sns.set_palette("viridis")

BASE_DIR = Path("..")
DATA_PROCESSED = BASE_DIR / "data" / "processed"
OUTPUTS = BASE_DIR / "outputs"
FIGURES = OUTPUTS / "figures"
SCENARIOS = OUTPUTS / "scenario_analysis"

# --- Load processed dataset and key JSONs ---
df = pd.read_csv(DATA_PROCESSED / "esg_processed_02b.csv")

with open(OUTPUTS / "model_evaluation_summary.json", "r") as f:
    model_summary = json.load(f)

with open(SCENARIOS / "scenario_metrics.json", "r") as f:
    scenario_metrics = json.load(f)

print("✅ Environment setup complete.")
print(f"Dataset shape: {df.shape}")

✅ Environment setup complete.
Dataset shape: (11000, 19)


In [2]:
# (Optional) Widget functionality test
import ipywidgets as widgets

print("✅ If the slider below appears and moves, widgets are working correctly.")
widgets.IntSlider()

✅ If the slider below appears and moves, widgets are working correctly.


IntSlider(value=0)

In [3]:
# %% ===============================================================
# STEP 1 — MODEL PERFORMANCE DASHBOARD
# ===============================================================
import json
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

# --- Load model evaluation summary (from Notebook 4) ---
SUMMARY_PATH = OUTPUTS / "model_evaluation_summary.json"

with open(SUMMARY_PATH, "r") as f:
    summary_metrics = json.load(f)

print(f"✅ Loaded model evaluation summary from: {SUMMARY_PATH}")

# --- Convert metrics into tidy DataFrames for plotting ---
# 1) Cross-validation performance
cv_df = (
    pd.DataFrame(summary_metrics["Cross-Validation ROC-AUC (mean)"], index=["Mean AUC"])
    .T.reset_index()
    .rename(columns={"index": "Model"})
)

# 2) Test-set performance
test_df = (
    pd.DataFrame(summary_metrics["Test ROC-AUC"], index=["ROC-AUC"])
    .T.reset_index()
    .rename(columns={"index": "Model"})
)

# 3) Thresholds and Brier scores
thr_df = pd.DataFrame(summary_metrics["Best Thresholds"], index=["Threshold"]).T
brier_df = pd.DataFrame(summary_metrics["Brier Score"], index=["Brier"]).T

# --- Combine results for dashboard summary ---
display(cv_df)
display(test_df)
display(thr_df)
display(brier_df)

# ===============================================================
# 📊 INTERACTIVE PLOTLY DASHBOARD — ROC-AUC COMPARISON
# ===============================================================
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=cv_df["Model"],
        y=cv_df["Mean AUC"],
        name="Cross-Validation AUC",
        marker_color="teal",
        opacity=0.7,
    )
)

fig.add_trace(
    go.Bar(
        x=test_df["Model"],
        y=test_df["ROC-AUC"],
        name="Test ROC-AUC",
        marker_color="goldenrod",
        opacity=0.7,
    )
)

fig.update_layout(
    title="Model Performance Comparison — ROC-AUC",
    xaxis_title="Model",
    yaxis_title="ROC-AUC",
    barmode="group",
    yaxis=dict(range=[0.5, 1.0]),
    template="plotly_white",
    width=800,
    height=500,
)

fig.show()

# ===============================================================
# 📊 OPTIONAL: GAUGE CHART FOR BEST MODEL PERFORMANCE
# ===============================================================
best_model = test_df.loc[test_df["ROC-AUC"].idxmax(), "Model"]
best_auc = test_df["ROC-AUC"].max()

fig_gauge = go.Figure(
    go.Indicator(
        mode="gauge+number",
        value=best_auc,
        title={"text": f"Best Model — {best_model}"},
        gauge={
            "axis": {"range": [0.5, 1.0]},
            "bar": {"color": "darkgreen"},
            "steps": [
                {"range": [0.5, 0.7], "color": "#f9e6e6"},
                {"range": [0.7, 0.85], "color": "#f0e68c"},
                {"range": [0.85, 1.0], "color": "#b6e3b6"},
            ],
        },
    )
)

fig_gauge.update_layout(width=500, height=400, template="plotly_white")
fig_gauge.show()

print("✅ STEP 1 — Model Performance Dashboard complete.")

✅ Loaded model evaluation summary from: ../outputs/model_evaluation_summary.json


,Model,Mean AUC
0,Logistic Regression,0.932042
1,Random Forest,0.922665
2,XGBoost,0.925595


,Model,ROC-AUC
0,Logistic Regression,0.925803
1,Random Forest,0.916101
2,XGBoost,0.920717


,Threshold
Balanced (max F1),0.366791
High Recall (≈ 0.90),0.222248
High Precision (≈ 0.90),0.714136


,Brier
Uncalibrated,0.111257
Calibrated,0.109477


✅ STEP 1 — Model Performance Dashboard complete.


In [4]:
# %% ===============================================================
# STEP 2 — CALIBRATION SUMMARY
# ===============================================================
import plotly.graph_objects as go
import plotly.express as px

# --- Extract calibration metrics ---
brier_uncal = summary_metrics["Brier Score"].get("Uncalibrated", None)
brier_cal = summary_metrics["Brier Score"].get("Calibrated", None)

print(f"🔹 Brier Score (Uncalibrated): {brier_uncal:.4f}")
print(f"🔹 Brier Score (Calibrated): {brier_cal:.4f}")

# --- Create a simple DataFrame for visualization ---
calib_df = pd.DataFrame({
    "Model": ["Uncalibrated", "Calibrated"],
    "Brier Score": [brier_uncal, brier_cal]
})

# ===============================================================
# 📊 Plotly Visualization — Calibration Improvement
# ===============================================================
fig = px.bar(
    calib_df,
    x="Model",
    y="Brier Score",
    text="Brier Score",
    title="Calibration Quality — Before vs After Calibration",
    color="Model",
    color_discrete_sequence=["#b56576", "#6a994e"]
)

fig.update_traces(texttemplate="%{text:.4f}", textposition="outside")
fig.update_layout(
    yaxis_title="Brier Score (lower is better)",
    xaxis_title="",
    yaxis=dict(range=[0, max(calib_df["Brier Score"]) * 1.2]),
    template="plotly_white",
    width=700,
    height=450
)
fig.show()

# ===============================================================
# 📈 Optional: Gauge indicator for calibration improvement
# ===============================================================
improvement = ((brier_uncal - brier_cal) / brier_uncal) * 100

fig_gauge_cal = go.Figure(
    go.Indicator(
        mode="gauge+number+delta",
        value=brier_cal,
        delta={"reference": brier_uncal, "valueformat": ".4f", "increasing": {"color": "red"}, "decreasing": {"color": "green"}},
        title={"text": f"Calibration Improvement ({improvement:.1f}% better)"},
        gauge={
            "axis": {"range": [0, max(brier_uncal * 1.5, 0.2)]},
            "bar": {"color": "#6a994e"},
            "steps": [
                {"range": [0, 0.1], "color": "#d9ead3"},
                {"range": [0.1, 0.15], "color": "#fce5cd"},
                {"range": [0.15, 0.2], "color": "#f4cccc"},
            ],
        },
        number={"valueformat": ".4f"},
    )
)
fig_gauge_cal.update_layout(width=500, height=400, template="plotly_white")
fig_gauge_cal.show()

print("✅ STEP 2 — Calibration Summary complete.")

🔹 Brier Score (Uncalibrated): 0.1113
🔹 Brier Score (Calibrated): 0.1095


✅ STEP 2 — Calibration Summary complete.


In [5]:
# %% ===============================================================
# STEP 3 — SCENARIO UPLIFT SUMMARY
# ===============================================================
import numpy as np
import plotly.express as px
from xgboost import XGBClassifier

# --- Load the processed dataset ---
DATA_PATH = DATA_PROCESSED / "esg_processed_02b.csv"
df = pd.read_csv(DATA_PATH)

# --- Prepare features and target ---
drop_cols = [
    "CompanyID", "CompanyName", "Industry", "Region", "Year",
    "ESG_Overall", "ESG_Overall_noisy", "ESG_Class"
]
X = df.drop(columns=drop_cols, errors="ignore")
y = df["ESG_Class"]

# --- Load or rebuild the best model (XGBoost) ---
xgb = XGBClassifier(
    n_estimators=300, learning_rate=0.05, max_depth=5,
    subsample=0.8, colsample_bytree=0.8,
    random_state=42, use_label_encoder=False, eval_metric="logloss"
)
xgb.fit(X, y)

# --- Define "what-if" uplift scenarios ---
scenarios = {
    "Baseline": X.copy(),
    "Governance +10%": X.assign(ESG_Governance=X["ESG_Governance"] * 1.10),
    "Social +10%": X.assign(ESG_Social=X["ESG_Social"] * 1.10),
    "Environmental +10%": X.assign(ESG_Environmental=X["ESG_Environmental"] * 1.10),
    "All ESG +10%": X.assign(
        ESG_Environmental=X["ESG_Environmental"] * 1.10,
        ESG_Social=X["ESG_Social"] * 1.10,
        ESG_Governance=X["ESG_Governance"] * 1.10
    ),
}

# --- Calculate uplift in ESG Class probability ---
uplift_results = []
for name, scenario in scenarios.items():
    prob = xgb.predict_proba(scenario)[:, 1].mean()
    uplift_results.append({"Scenario": name, "Avg_Prob_High_ESG": prob})

uplift_df = pd.DataFrame(uplift_results)
uplift_df["Uplift_vs_Baseline"] = uplift_df["Avg_Prob_High_ESG"] - uplift_df.loc[uplift_df["Scenario"] == "Baseline", "Avg_Prob_High_ESG"].values[0]

display(uplift_df)

# ===============================================================
# 📊 PLOTLY VISUALIZATION — Scenario Uplift
# ===============================================================
fig = px.bar(
    uplift_df.sort_values("Avg_Prob_High_ESG"),
    x="Scenario",
    y="Avg_Prob_High_ESG",
    text="Uplift_vs_Baseline",
    title="Scenario Uplift — Impact of ESG Improvements on High ESG Probability",
    color="Scenario",
    color_discrete_sequence=px.colors.qualitative.Vivid
)

fig.update_traces(texttemplate="+%{text:.3f}", textposition="outside")
fig.update_layout(
    yaxis_title="Avg Predicted Probability (High ESG)",
    xaxis_title="Scenario",
    yaxis=dict(range=[0, 1]),
    template="plotly_white",
    width=800,
    height=500
)
fig.show()

print("✅ STEP 3 — Scenario Uplift Summary complete.")

,Scenario,Avg_Prob_High_ESG,Uplift_vs_Baseline
0,Baseline,0.386001,0.000000
1,Governance +10%,0.447944,0.061942
2,Social +10%,0.450265,0.064264
3,Environmental +10%,0.365766,-0.020235
4,All ESG +10%,0.493121,0.107120


✅ STEP 3 — Scenario Uplift Summary complete.


In [6]:
# %% ===============================================================
# STEP 4 — INDUSTRY / REGION IMPACT COMPARISON
# ===============================================================
import plotly.express as px
import plotly.graph_objects as go

# --- Reload dataset for grouping clarity ---
df = pd.read_csv(DATA_PATH)

# --- Prepare model inputs (same features as before) ---
X = df.drop(columns=[
    "CompanyID","CompanyName","Industry","Region","Year",
    "ESG_Overall","ESG_Overall_noisy","ESG_Class"
], errors="ignore")

# --- Predict High ESG probability for each company ---
df["Pred_Prob_High_ESG"] = xgb.predict_proba(X)[:, 1]

# ===============================================================
# 📊 4.1 INDUSTRY-LEVEL AVERAGE ESG PROBABILITIES
# ===============================================================
industry_avg = (
    df.groupby("Industry")["Pred_Prob_High_ESG"]
    .mean()
    .sort_values(ascending=False)
    .reset_index()
)

fig_industry = px.bar(
    industry_avg,
    x="Pred_Prob_High_ESG",
    y="Industry",
    orientation="h",
    text="Pred_Prob_High_ESG",
    color="Pred_Prob_High_ESG",
    color_continuous_scale="viridis",
    title="Average Predicted High ESG Probability by Industry"
)

fig_industry.update_traces(texttemplate="%{text:.2f}", textposition="outside")
fig_industry.update_layout(
    xaxis_title="Average Probability (High ESG)",
    yaxis_title="Industry",
    coloraxis_showscale=False,
    height=600,
    width=800,
    template="plotly_white"
)
fig_industry.show()

# ===============================================================
# 📊 4.2 REGION-LEVEL AVERAGE ESG PROBABILITIES
# ===============================================================
region_avg = (
    df.groupby("Region")["Pred_Prob_High_ESG"]
    .mean()
    .sort_values(ascending=False)
    .reset_index()
)

fig_region = px.bar(
    region_avg,
    x="Region",
    y="Pred_Prob_High_ESG",
    text="Pred_Prob_High_ESG",
    color="Pred_Prob_High_ESG",
    color_continuous_scale="Blues",
    title="Average Predicted High ESG Probability by Region"
)

fig_region.update_traces(texttemplate="%{text:.2f}", textposition="outside")
fig_region.update_layout(
    xaxis_title="Region",
    yaxis_title="Average Probability (High ESG)",
    coloraxis_showscale=False,
    height=500,
    width=800,
    template="plotly_white"
)
fig_region.show()

# ===============================================================
# 📈 4.3 INDUSTRY × REGION HEATMAP
# ===============================================================
heatmap_df = (
    df.groupby(["Industry","Region"])["Pred_Prob_High_ESG"]
    .mean()
    .reset_index()
)

fig_heatmap = px.density_heatmap(
    heatmap_df,
    x="Region",
    y="Industry",
    z="Pred_Prob_High_ESG",
    color_continuous_scale="viridis",
    title="Heatmap — ESG Probability by Industry and Region",
    nbinsx=len(heatmap_df["Region"].unique()),
    nbinsy=len(heatmap_df["Industry"].unique())
)

fig_heatmap.update_layout(
    width=900,
    height=600,
    template="plotly_white"
)
fig_heatmap.show()

print("✅ STEP 4 — Industry / Region Impact Comparison complete.")

✅ STEP 4 — Industry / Region Impact Comparison complete.


In [7]:
# %% ===============================================================
# STEP 5 — INTERACTIVE SCENARIO DASHBOARD (PLOTLY)
# ===============================================================
import ipywidgets as widgets
from IPython.display import display
import plotly.graph_objects as go

# --- Helper function: predict probability from slider inputs ---
def predict_esg_prob(env, soc, gov, rev, pm, mc, gr):
    """Predicts probability of being High ESG given user inputs."""

    # Construct input dataframe with all expected columns
    input_df = pd.DataFrame([{
        "Revenue": rev,
        "ProfitMargin": pm,
        "MarketCap": mc,
        "GrowthRate": gr,
        "ESG_Environmental": env,
        "ESG_Social": soc,
        "ESG_Governance": gov,
        "CarbonEmissions": df["CarbonEmissions"].mean(),
        "WaterUsage": df["WaterUsage"].mean(),
        "EnergyConsumption": df["EnergyConsumption"].mean(),
        "CarbonEmissions_log": np.log1p(df["CarbonEmissions"].mean())
    }])

    # ✅ Reorder columns to match the training data
    input_df = input_df[X.columns]

    return xgb.predict_proba(input_df)[0, 1]

# --- Define interactive sliders ---
env_slider = widgets.FloatSlider(value=50, min=0, max=100, step=1, description='Environmental', continuous_update=False)
soc_slider = widgets.FloatSlider(value=50, min=0, max=100, step=1, description='Social', continuous_update=False)
gov_slider = widgets.FloatSlider(value=50, min=0, max=100, step=1, description='Governance', continuous_update=False)
rev_slider = widgets.FloatSlider(value=500, min=100, max=2000, step=50, description='Revenue', continuous_update=False)
pm_slider  = widgets.FloatSlider(value=5, min=-10, max=30, step=1, description='ProfitMargin', continuous_update=False)
mc_slider  = widgets.FloatSlider(value=1000, min=100, max=10000, step=100, description='MarketCap', continuous_update=False)
gr_slider  = widgets.FloatSlider(value=5, min=-20, max=40, step=1, description='GrowthRate', continuous_update=False)

# --- Interactive update function ---
def update_dashboard(env, soc, gov, rev, pm, mc, gr):
    prob = predict_esg_prob(env, soc, gov, rev, pm, mc, gr)

    # Gauge visualization
    fig = go.Figure(go.Indicator(
        mode="gauge+number",
        value=prob * 100,
        title={'text': "Predicted Probability: High ESG (%)"},
        gauge={
            'axis': {'range': [0, 100]},
            'bar': {'color': "#6a994e"},
            'steps': [
                {'range': [0, 50], 'color': "#f4cccc"},
                {'range': [50, 75], 'color': "#ffe599"},
                {'range': [75, 100], 'color': "#b6d7a8"}
            ]
        }
    ))

    fig.update_layout(width=500, height=400, template="plotly_white")
    fig.show()

    print(f"🔹 Predicted Probability (High ESG): {prob:.3f}")

# --- Display interactive dashboard ---
ui = widgets.VBox([
    widgets.HTML("<h3 style='color:#4b5563'>Interactive ESG Scenario Simulator</h3>"),
    env_slider, soc_slider, gov_slider,
    rev_slider, pm_slider, mc_slider, gr_slider
])

out = widgets.interactive_output(update_dashboard, {
    'env': env_slider,
    'soc': soc_slider,
    'gov': gov_slider,
    'rev': rev_slider,
    'pm': pm_slider,
    'mc': mc_slider,
    'gr': gr_slider
})

display(ui, out)

Output()

In [8]:
# %% ===============================================================
# STEP 6 — EXPORT DASHBOARD FIGURES
# ===============================================================
import plotly.io as pio

# Create directory if not present
FIGURES.mkdir(parents=True, exist_ok=True)

# --- Export key Plotly figures from earlier steps ---
try:
    # Save static PNG versions
    fig.write_image(FIGURES / "interactive_dashboard_gauge.png", scale=2)
    fig_industry.write_image(FIGURES / "industry_avg_esg_prob.png", scale=2)
    fig_region.write_image(FIGURES / "region_avg_esg_prob.png", scale=2)
    fig_heatmap.write_image(FIGURES / "industry_region_heatmap.png", scale=2)
    print("✅ All static figures exported as PNG.")
except Exception as e:
    print(f"⚠️ PNG export issue: {e}")

# --- Optional: export as interactive HTML files (fully explorable offline) ---
try:
    pio.write_html(fig_industry, FIGURES / "industry_avg_esg_prob.html", auto_open=False)
    pio.write_html(fig_region, FIGURES / "region_avg_esg_prob.html", auto_open=False)
    pio.write_html(fig_heatmap, FIGURES / "industry_region_heatmap.html", auto_open=False)
    print("✅ Interactive HTML dashboards exported.")
except Exception as e:
    print(f"⚠️ HTML export issue: {e}")

print("💾 STEP 6 — Export Dashboard Figures complete.")

⚠️ PNG export issue: 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido

✅ Interactive HTML dashboards exported.
💾 STEP 6 — Export Dashboard Figures complete.


---

### 🎯 Project Conclusion

This project demonstrated end-to-end classification modeling using a simulated but realistic ESG dataset.  
Key steps included:
- Data preparation with controlled noise and correlation
- Training and evaluating Logistic Regression, Random Forest, and XGBoost
- Model interpretability via calibration, SHAP, and partial dependence
- Building an interactive dashboard for ESG scenario simulation

**Main Takeaways:**
- Logistic Regression provides strong interpretability.
- XGBoost achieved the highest AUC after proper noise introduction and tuning.
- Calibration and threshold tuning are essential for probabilistic models.
- Model transparency (via SHAP) bridges the gap between ML and ESG decision-making.

This notebook series can serve as a reusable framework for learning, experimentation, and communication of classification models.